In [1]:
import sleap_io as sio
import xarray as xr

import movement.kinematics as kin
from movement import sample_data
import numpy as np
ds = sample_data.fetch_dataset(
    "DLC_rotating-mouse_eye-tracking_stim-black.predictions.h5",
    with_video=True)


        
ds["velocity"] = kin.compute_velocity(ds.position)

eye_midpoint = ds.position.sel(keypoints=["eye-L", "eye-R"]).mean("keypoints")
positions_norm =  ds.position - eye_midpoint
pupil_centroid = (
    positions_norm.sel(keypoints=["pupil-L", "pupil-R"])
    .mean("keypoints")
    .assign_coords({"keypoints": "pupil-C"})
)

positions_norm = xr.concat([positions_norm, pupil_centroid], dim="keypoints")

pupil_velocity = kin.compute_velocity(positions_norm.sel(keypoints="pupil-C"))
pupil_velocity.name = "pupil velocity"

pupil_diameter: xr.DataArray = kin.compute_pairwise_distances(
    positions_norm, dim="keypoints", pairs={"pupil-L": "pupil-R"}
)
pupil_diameter.name = "pupil diameter"

distance_between_eye_keypoints: xr.DataArray = kin.compute_pairwise_distances(
    positions_norm, dim="keypoints", pairs={"eye-L": "eye-R"}
)
distance_between_eye_keypoints.name = "distance (eye-L - eye-R)"


ds["position_norm"] = positions_norm
ds["pupil_velocity"] = pupil_velocity
ds["pupil_diameter"] = pupil_diameter
ds["distance_eyeL_to_eyeR"] = distance_between_eye_keypoints

ds = ds.expand_dims(trials=[0])
for var in ['position', 'position_norm', 'velocity', 'pupil_velocity', 'pupil_diameter', 'distance_eyeL_to_eyeR']:
    if 'individuals' not in ds[var].dims:
        ds[var] = ds[var].expand_dims(individuals=["individual_0"])
        


output_path = r"C:\Users\Admin\Documents\Akseli\sample_data\eye\dlc_eye_tracking.nc"
ds["cam1"] = ('trials', ['video1.mp4'])
ds["cam1"].attrs["type"] = "cameras"
ds["dlc1"] = ('trials', ['tracking1.h5'])
ds["dlc1"].attrs["type"] = "tracking"


ds["position"].attrs["type"] = "features"
ds["position_norm"].attrs["type"] = "features"
ds["velocity"].attrs["type"] = "features"
ds["pupil_velocity"].attrs["type"] = "features"
ds["pupil_diameter"].attrs["type"] = "features"
ds["distance_eyeL_to_eyeR"].attrs["type"] = "features"



shape = (1, int(ds.dims["time"]))
ds["labels"] = (('trials', 'time'), np.zeros(shape))
ds.to_netcdf(output_path)

SHA256 hash of downloaded file: 22100193c76c0e3f274e4c986357c95fc645da1f5c67af41a8acfa91313205f4
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
2025-09-11 17:23:31.103 | WARNING  | xarray.core.utils:emit_user_level_warning:1194 - C:\Users\Admin\AppData\Local\Temp\ipykernel_17496\1686208133.py:67: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  shape = (1, int(ds.dims["time"]))



In [4]:
ds["velocity"]

<xarray.DataArray 'velocity' (trials: 1, time: 7000, space: 2, keypoints: 4,
                              individuals: 1)> Size: 448kB
0.957 6.172 26.51 -36.35 52.14 44.87 ... 15.3 -10.28 -11.25 15.85 -32.58 55.59
Coordinates:
  * trials       (trials) int64 8B 0
  * time         (time) float64 56kB 0.0 0.025 0.05 0.075 ... 174.9 174.9 175.0
  * space        (space) <U1 8B 'x' 'y'
  * keypoints    (keypoints) <U7 112B 'pupil-L' 'pupil-R' 'eye-L' 'eye-R'
  * individuals  (individuals) object 8B 'individual_0'
Attributes:
    type:     features

In [24]:
ds["position_norm"]

<xarray.DataArray 'position_norm' (trials: 1, time: 7000, space: 2,
                                   keypoints: 4, individuals: 1)> Size: 448kB
-25.56 77.44 -134.8 134.8 4.137 15.51 ... -147.7 147.7 -4.334 17.83 -30.61 30.61
Coordinates:
  * trials       (trials) int64 8B 0
  * time         (time) float64 56kB 0.0 0.025 0.05 0.075 ... 174.9 174.9 175.0
  * space        (space) <U1 8B 'x' 'y'
  * keypoints    (keypoints) <U7 112B 'pupil-L' 'pupil-R' 'eye-L' 'eye-R'
  * individuals  (individuals) <U12 48B 'individual_0'
Attributes:
    type:     features

In [15]:
pupil_centroid = (
    positions_norm.sel(keypoints=["pupil-L", "pupil-R"])
    .mean("keypoints")
    .assign_coords({"keypoints": "pupil-C"})
)
pupil_centroid

<xarray.DataArray 'position' (trials: 1, time: 7000, space: 2, individuals: 1)> Size: 112kB
25.94 9.826 26.15 10.92 25.45 10.19 25.84 ... 25.92 7.605 25.47 6.976 25.9 6.746
Coordinates:
  * trials       (trials) int64 8B 0
  * time         (time) float64 56kB 0.0 0.025 0.05 0.075 ... 174.9 174.9 175.0
  * space        (space) <U1 8B 'x' 'y'
  * individuals  (individuals) <U12 48B 'individual_0'
    keypoints    <U7 28B 'pupil-C'
Attributes:
    type:     features

In [11]:
np.zeros_like(ds.velocity.values).shape

(7000, 2, 4, 1)

In [4]:
ds

<xarray.Dataset> Size: 1MB
Dimensions:      (time: 7000, space: 2, keypoints: 4, individuals: 1, trials: 1)
Coordinates:
  * time         (time) float64 56kB 0.0 0.025 0.05 0.075 ... 174.9 174.9 175.0
  * space        (space) <U1 8B 'x' 'y'
  * keypoints    (keypoints) <U7 112B 'pupil-L' 'pupil-R' 'eye-L' 'eye-R'
  * individuals  (individuals) <U12 48B 'individual_0'
  * trials       (trials) int64 8B 0
Data variables:
    position     (time, space, keypoints, individuals) float64 448kB 258.8 .....
    confidence   (time, keypoints, individuals) float64 224kB 1.0 1.0 ... 1.0
    velocity     (time, space, keypoints, individuals) float64 448kB 0.957 .....
    cam1         (trials) <U10 40B 'video1.mp4'
    dlc1         (trials) <U12 48B 'tracking1.h5'
    labels       (trials, time) float64 56kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    source_software:  DeepLabCut
    ds_type:          poses
    fps:              40.0
    time_unit:        seconds
    source_file:      C:/Users/Admin/.movement/data/poses/DLC_rotating-mouse_...
    frame_path:       C:/Users/Admin/.movement/data/frames/rotating-mouse_eye...
    video_path:       C:/Users/Admin/.movement/data/videos/rotating-mouse_eye...

In [ ]:
ds.video_path



'C:/Users/Admin/.movement/data/videos/rotating-mouse_eye-tracking_stim-black_video.mp4'

In [ ]:
from datashuttle import DataShuttle


project = DataShuttle("AI_data")
project.upload_custom(
    top_level_folder="derivatives",
    sub_names="all_sub",
    ses_names="ses-000_date-20250309",
    datatype="behav",
    dry_run=True,
)


: 